# FEA Solver Element List

This guide contains documentation and theory for each structural element available in the MATLAB FEA solver, as coded in the script FEA_Solver.m and its supporting functions. This document is meant to provide a reference for the theory behind each finite element type used in the code, as well as concretely how they should be defined in the solver input file. 

**NOTE:** This document needs to be updated as soon as any new element capabilities are defined in the main solver code, so that no confusion arises from lack of proper documentation.

## Table of Contents
* [Input File Syntax](#Input_File_Syntax)
* [List of Elements](#List_of_Elements)
    * [BAR2D](#BAR2D)
    * [BEAM2D](#BEAM2D)
    * [BEAM3D](#BEAM3D)
    * [2DSTRA](#2DSTRA)

## Input File Syntax <a class="anchor" id="Input_File_Syntax"></a>

The solver input file is the starting point of any FEA analysis using this code, in which all relevant information will be defined. The code will throw errors for any obvious mistakes, such as defining an incorrect number of nodes to each element, but more subtle errors such as incorrect node ordering will be passed through the code and produce erroneous results. As such, it is vital to get the input file right first time, especially if the mesh is large and matrix inversion may take a while. Firstly, it is essential that the input file has a .txt extension, as the MATLAB function used to parse through the text cannot handle other file types. The data in the file is read line-by-line and fed into appropriate matrices containing the node, element, force and boundary condition information for the problem, delimited horizontally by spaces. As such, the input file will have the following overall syntax:

In [1]:
# **nodes
# 1 0.0 0.0  # node number, x-coordinate, y-coordinate, (z-coordinate)
# 2 0.0 1.0
# ...
# **elements
# 1 100 1 ... 1e11 ...  # element number, element ID, nodes, element properties
# 2 201 1 ... 1e11 ...
# ...
# **bcs
# 1 1 0.0  # node number, degree of freedom, displacement constraint
# 1 2 0.0
# ...
# **forces
# 2 1 100.0  # node number, degree of freedom, force value
# 2 2 100.0
# ...

#### Nodes

The node definitions are relatively straightforward. The nodes are each defined by a unique identifier number (the "node number" referenced everywhere else in the text), and then the x- and y-coordinates of the node are quoted afterwards in succession. The z-coordinate of the node is unnecessary in 2D problems, so it may be omitted in such cases. In order for the solver to produce meaningful results, each node must be unique: no two nodes can share the same coordinates.

#### Elements

The element definitions are the object of the subsequent parts of this document, and as such the details for each different element type can be found under the appropriate headings. However, the element definitions generally follow the syntax specified above in the specimen input file:

In [2]:
# element number, element ID, node numbers, element properties

The element number has much the same function as the node number above, and each element must similarly have a unique element number. The element ID is a 3-digit figure which specifies the element type (as listed below). There can then be as many node numbers as required for the chosen element type, and finally the element properties, which combine its material and geometrical characteristics.

#### Boundary Conditions

The boundary condition definitions are again relatively straightforward. Firstly, the file specifies the node on which the boundary condition is to be applied, then the relevant degree of freedom. Note that the numbers used to label each of the degrees of freedom will depend again on the elements being used in the solution, since each element type can define different numbers of degrees of freedom at the nodes. Finally, the displacement constraint value is specified (for rotational degrees of freedom, this must be quoted in radians).

#### Forces

The force definitions work in much the same way as the boundary condition definitions. However, instead of a displacement value in the third slot, here a force must be provided. Importantly, this cannot be a magnitude, and the sign of the force must be included.

## List of Elements <a id="List_of_Elements"></a>

### BAR2D <a id="BAR2D"></a>

Element ID: 100. 2D bar element, allowing only uniaxial tension and compression along the element's length.

#### Input Syntax

This element has two nodes, with a single degree of freedom at each node. (However, note that this increases to two nodal degrees of freedom when the element stiffness matrix is translated into global coordinates, so the DOF numbering is \[x=1, y=2\].) The Young's modulus $E$ and cross-sectional area $A$ must be provided, so the input syntax is:

In [ ]:
# e 100 n1 n2 E A

#### Theory

In element coordinates, the element stiffness matrix is 
$$
\mathbf{k}^e = \frac{EA}{L}\left[\begin{matrix}
1 & -1 \\
-1 & 1
\end{matrix}\right]
$$
and the length and angle of the element with respect to the global coordinates can be found from the figures above by:
$$
L = \sqrt{(x_j-x_i)^2+(y_j-y_i)^2}
$$
$$
\alpha = \arctan{\left(\frac{y_j-y_i}{x_j-x_i}\right)}
$$
Thus, denoting $\cos{\alpha}=c$ and $\sin{\alpha}=s$, the element stiffness matrix in global coordinates is:
$$
\mathbf{K}^e = \left[\begin{matrix}
c^2 & cs & -c^2 & -cs \\
cs & s^2 & -cs & -s^2 \\
-c^2 & -cs & c^2 & cs \\
-cs & -s^2 & cs & s^2
\end{matrix}\right]
$$

### BEAM2D <a id="BEAM2D"></a>

Element ID: 101. 2D beam element, allowing uniaxial tension and compression, deflection, and rotation in a single plane.

#### Input Syntax

This element has two nodes, and three degrees of freedom at each node (\[x=1, y=2, $\theta$=3\]). Here, the provided information must include the Young's modulus $E$, cross-sectional area $A$, and second moment of cross-sectional area $I$. The input syntax is:

In [ ]:
# e 101 n1 n2 E A I

#### Theory

In element coordinates, the element stiffness matrix is:
$$
\mathbf{k}^e = \left[\begin{matrix}
\frac{EA}{L} & 0 & 0 & -\frac{EA}{L} & 0 & 0 \\
0 & \frac{12EI}{L^3} & \frac{6EI}{L^2} & 0 & -\frac{12EI}{L^3} & \frac{6EI}{L^2} \\
0 & \frac{6EI}{L^2} & \frac{4EI}{L} & 0 & -\frac{6EI}{L^2} & \frac{2EI}{L} \\
-\frac{EA}{L} & 0 & 0 & \frac{EA}{L} & 0 & 0 \\
0 & -\frac{12EI}{L^3} & -\frac{6EI}{L^2} & 0 & \frac{12EI}{L^3} & -\frac{6EI}{L^2} \\
0 & \frac{6EI}{L^2} & \frac{2EI}{L} & 0 & -\frac{6EI}{L^2} & \frac{4EI}{L}
\end{matrix}\right]
$$
The length and angle of the element from the global coordinate system are found by:
$$
L = \sqrt{(x_j-x_i)^2+(y_j-y_i)^2}
$$
$$
\alpha = \arctan{\left(\frac{y_j-y_i}{x_j-x_i}\right)}
$$
so that the transformation matrix from the element coordinates to the global coordinates is:
$$
\mathbf{T} = \left[\begin{matrix}
\cos{\alpha} & \sin{\alpha} & 0 & 0 & 0 & 0 \\
-\sin{\alpha} & \cos{\alpha} & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & \cos{\alpha} & \sin{\alpha} & 0 \\
0 & 0 & 0 & -\sin{\alpha} & \cos{\alpha} & 0 \\
0 & 0 & 0 & 0 & 0 & 1
\end{matrix}\right]
$$
The final element stiffness matrix in the global coordinates is then given by:
$$
\mathbf{K}^e = \mathbf{T}^T\mathbf{k}^e\mathbf{T}
$$

### BEAM3D <a id="BEAM3D"></a>

Element ID: 102. 3D beam element, allowing forces and moments in all three principal directions.

#### Input Syntax

This element has two nodes, with six degrees of freedom defined at each node: \[x=1, y=2, z=3, $\theta_x$=4, $\theta_y$=5, $\theta_z$=6\]. The element properties provided in the definition must include the Young's modulus $E$, the Poisson's ratio $\nu$, the cross-sectional area $A$, the second moment of area about the y-axis $I_y$, the second moment of area about the z-axis $I_z$, the torsional constant $J$, and the three components (in the global coordinate system) of a vector in the element's y-direction. Thus, the input syntax is:

In [ ]:
# e 102 n1 n2 E nu A Iy Iz J y1 y2 y3

#### Theory

In element coordinates, the element stiffness matrix is:
$$
\mathbf{k}^e = \left[\begin{matrix}
\frac{EA}{L} & 0 & 0 & 0 & 0 & 0 & -\frac{EA}{L} & 0 & 0 & 0 & 0 & 0 \\
0 & \frac{12EI_z}{L^3} & 0 & 0 & 0 & \frac{6EI_z}{L^2} & 0 & -\frac{12EI_z}{L^3} & 0 & 0 & 0 & \frac{6EI_z}{L^2} \\
0 & 0 & \frac{12EI_y}{L^3} & 0 & -\frac{6EI_y}{L^2} & 0 & 0 & 0 & -\frac{12EI_y}{L^3} & 0 & -\frac{6EI_y}{L^2} & 0 \\
0 & 0 & 0 & \frac{GJ}{L} & 0 & 0 & 0 & 0 & 0 & -\frac{GJ}{L} & 0 & 0 \\
0 & 0 & -\frac{6EI_y}{L^2} & 0 & \frac{4EI_y}{L} & 0 & 0 & 0 & \frac{6EI_y}{L^2} & 0 & \frac{2EI_y}{L} & 0 \\
0 & \frac{6EI_z}{L^2} & 0 & 0 & 0 & \frac{4EI_z}{L} & 0 & -\frac{6EI_z}{L^2} & 0 & 0 & 0 & \frac{2EI_z}{L} \\
-\frac{EA}{L} & 0 & 0 & 0 & 0 & 0 & \frac{EA}{L} & 0 & 0 & 0 & 0 & 0 \\
0 & -\frac{12EI_z}{L^3} & 0 & 0 & 0 & -\frac{6EI_z}{L^2} & 0 & \frac{12EI_z}{L^3} & 0 & 0 & 0 & -\frac{6EI_z}{L^2} \\
0 & 0 & -\frac{12EI_y}{L^3} & 0 & \frac{6EI_y}{L^2} & 0 & 0 & 0 & \frac{12EI_y}{L^3} & 0 & \frac{6EI_y}{L^2} & 0 \\
0 & 0 & 0 & -\frac{GJ}{L} & 0 & 0 & 0 & 0 & 0 & \frac{GJ}{L} & 0 & 0 \\
0 & 0 & -\frac{6EI_y}{L^2} & 0 & \frac{2EI_y}{L} & 0 & 0 & 0 & \frac{6EI_y}{L^2} & 0 & \frac{4EI_y}{L} & 0 \\
0 & \frac{6EI_z}{L^2} & 0 & 0 & 0 & \frac{2EI_z}{L} & 0 & -\frac{6EI_z}{L^2} & 0 & 0 & 0 & \frac{4EI_z}{L}
\end{matrix}\right]
$$

In order to find the transformation matrix, we need the direction cosines of each of the element coordinate axes with each of the global coordinate axes, which are computed as:

|    | $\bar{x}$| $\bar{y}$| $\bar{z}$|
| --- | --- | --- | --- |
|$x$| $l_1$    | $m_1$   | $n_1$    |
|$y$| $l_2$    | $m_2$   | $n_2$    |
|$z$| $l_3$    | $m_3$   | $n_3$    |

where the direction cosines are defined as $l_1=\cos{(x,\bar{x})}$, $m_1=\cos{(x,\bar{y})}$, $n_1=\cos{(x,\bar{z})}$, etc., and for a general vector $\mathbf{v}$,
$$
\cos{(\mathbf{v},\mathbf{e}_i)} = \frac{\mathbf{v}\cdot\mathbf{e}_i}{||\mathbf{v}||}
$$
Then, the transformation matrix for a single node $p$ is:
$$
\mathbf{T}_p = \left[\begin{matrix}
l_1 & m_1 & n_1 & 0 & 0 & 0 \\
l_2 & m_2 & n_2 & 0 & 0 & 0 \\
l_3 & m_3 & n_3 & 0 & 0 & 0 \\
0 & 0 & 0 & l_1 & m_1 & n_1 \\
0 & 0 & 0 & l_2 & m_2 & n_2 \\
0 & 0 & 0 & l_3 & m_3 & n_3
\end{matrix}\right]
$$
and the full transformation matrix for element $i$ between nodes $i$ and $j$ is:
$$
\mathbf{T} = \left[\begin{matrix}
\left[\mathbf{T}_i\right] & \left[\mathbf{0}\right] \\
\left[\mathbf{0}\right] & \left[\mathbf{T}_j\right]
\end{matrix}\right]
$$
Finally, the element stiffness matrix in the global coordinates is given by
$$
\mathbf{K}^e = \mathbf{T}^T\mathbf{k}^e\mathbf{T}
$$

### 2DSTRA <a id="2DSTRA"></a>

Element ID: 201. 2D plane-strain quadrilateral element.

#### Input Syntax

This element has four nodes, each with two degrees of freedom: \[x=1, y=1\]. The necessary element properties here are only the Young's modulus $E$ and the Poisson's ratio $\nu$, giving the following input syntax:

In [3]:
# e 201 n1 n2 n3 n4 E nu

Note that the nodes must be ordered in an anticlockwise direction. It is unimportant which node exactly is used to start the definition (i.e. which node is chosen as $n_1$), but the resulting definition must traverse the element perimeter in an anticlockwise fashion.

#### Theory

For finite-element calculations, the principle of virtual work - expressed in terms of stress and strain for each element $m$ - is written as
$$
\sum_m\int_{V^{(m)}}\delta\varepsilon^T\sigma dV = \sum_m\int_{V^{(m)}}\delta\mathbf{u}^T\mathbf{b} dV + \sum_m\int_{A^{(m)}}\delta\mathbf{u}^T\mathbf{t} dA + \sum\delta\mathbf{u}^T\mathbf{f}
$$
with body forces $\mathbf{b}$, traction forces $\mathbf{t}$, and nodal forces $\mathbf{f}$.

The key step in solving this equation is to discretise and use isoparametric shape functions. Introduce a coordinate mapping $N$, taking a general quadrilateral element to an origin-centred square.

For the case of linear plane-strain quadrilateral elements, one way to perform this mapping is to introduce the following shape functions from natural coordinates $(\xi,\eta)$ to ordinary coordinates $(x,y)$:
$$
N_1 = \frac{1}{4}(1-\xi)(1-\eta)
$$
$$
N_2 = \frac{1}{4}(1+\xi)(1-\eta)
$$
$$
N_3 = \frac{1}{4}(1+\xi)(1+\eta)
$$
$$
N_4 = \frac{1}{4}(1-\xi)(1+\eta)
$$
If the locations of the nodes are known in ordinary coordinates, i.e. $(x_i,y_i)$, and there is a target location $(\xi,\eta)$ in natural coordinates, then the corresponding ordinary coordinates are
$$
x = N_1(\xi,\eta)x_1 + N_2(\xi,\eta)x_2 + N_3(\xi,\eta)x_3 + N_4(\xi,\eta)x_4
$$
$$
y = N_1(\xi,\eta)y_1 + N_2(\xi,\eta)y_2 + N_3(\xi,\eta)y_3 + N_4(\xi,\eta)y_4
$$
This formulation also works for displacements:
$$
u = N_i(\xi,\eta)u_i, \quad v = N_i(\xi,\eta)v_i
$$
With the displacements found, the strains are given by:
$$
\varepsilon_{ij} = \left[\begin{matrix}
\varepsilon_{11} \\
\varepsilon_{22} \\
\varepsilon_{12}
\end{matrix}\right] = \left[\begin{matrix}
\partial u/\partial x \\
\partial v/\partial y \\
\frac{1}{2}\left(\frac{\partial u}{\partial y}+\frac{\partial v}{\partial x}\right)
\end{matrix}\right]
$$
Defining the Jacobian $\mathbf{J}$ of the shape functions:
$$
\left[\begin{matrix}
\frac{\partial u}{\partial\xi} \\
\frac{\partial u}{\partial\eta}
\end{matrix}\right] = \left[\begin{matrix}
\frac{\partial x}{\partial\xi} & \frac{\partial y}{\partial\xi} \\
\frac{\partial x}{\partial\eta} & \frac{\partial y}{\partial\eta}
\end{matrix}\right]\left[\begin{matrix}
\frac{\partial u}{\partial x} \\
\frac{\partial u}{\partial y}
\end{matrix}\right] = \left[\begin{matrix}
J_{11} & J_{12} \\
J_{21} & J_{22}
\end{matrix}\right]\left[\begin{matrix}
\frac{\partial u}{\partial x} \\
\frac{\partial u}{\partial y}
\end{matrix}\right]
$$

With the nodal coordinates collected in the following matrix:
$$
\mathbf{x} = \left[\begin{matrix}
x_1 & x_2 & x_3 & x_4 \\
y_1 & y_2 & y_3 & y_4
\end{matrix}\right]^T
$$
and the gradient of the shape functions defined as
$$
\nabla\mathbf{N}(\xi,\eta) = \left[\begin{matrix}
\partial N_1/\partial\xi & \partial N_2/\partial\xi & \partial N_3/\partial\xi & \partial N_4/\partial\xi \\
\partial N_1/\partial\eta & \partial N_2/\partial\eta & \partial N_3/\partial\eta & \partial N_4/\partial\eta
\end{matrix}\right]
$$
the Jacobian is given as $\mathbf{J} = \nabla\mathbf{N}\mathbf{x}$. Then, the partial derivatives in the ordinary coordinates can be written as
$$
\left[\begin{matrix}
\frac{\partial u}{\partial x} \\
\frac{\partial u}{\partial y}
\end{matrix}\right] = \mathbf{J}^{-1}\nabla\mathbf{N}\left[\begin{matrix}
u_1 & u_2 & u_3 & u_4
\end{matrix}\right]^T
$$
$$
\left[\begin{matrix}
\frac{\partial v}{\partial x} \\
\frac{\partial v}{\partial y}
\end{matrix}\right] = \mathbf{J}^{-1}\nabla\mathbf{N}\left[\begin{matrix}
v_1 & v_2 & v_3 & v_4
\end{matrix}\right]^T
$$

Defining $\mathbf{M}=\mathbf{J}^{-1}\nabla\mathbf{N}$, the strains are:
$$
\left[\begin{matrix}
\varepsilon_{11} \\
\varepsilon_{22} \\
\varepsilon_{12}
\end{matrix}\right] = \left[\begin{matrix}
M_{11} & 0 & M_{12} & 0 & M_{13} & 0 & M_{14} & 0 \\
0 & M_{21} & 0 & M_{22} & 0 & M_{23} & 0 & M_{24} \\
M_{11} & M_{21} & M_{12} & M_{22} & M_{13} & M_{23} & M_{14} & M_{24}
\end{matrix}\right]\left[\begin{matrix}
u_1 \\ v_1 \\ u_2 \\ v_2 \\ u_3 \\ v_3 \\u_4 \\ v_4
\end{matrix}\right]
$$
$$
\varepsilon^{(m)} = \mathbf{B}^{(m)}\mathbf{U}^{(m)}
$$

With the strain found, Hooke's law gives the stress as:
$$
\sigma^{(m)} = \mathbf{C}^{(m)}\varepsilon^{(m)} = \mathbf{C}^{(m)}\mathbf{B}^{(m)}\mathbf{U}^{(m)}
$$
where the substantive matrix is
$$
\mathbf{C}^{(m)} = \frac{E}{(1+\nu)(1-2\nu)}\left[\begin{matrix}
1-\nu & \nu & 0 \\
\nu & 1-\nu & 0 \\
0 & 0 & \frac{1}{2}-\nu
\end{matrix}\right]
$$
Using Gaussian quadrature to evaluate the first integral in the principle of virtual work, and ignoring the body and traction forces, the final formulation is then
$$
\left[\sum_p^{np}\mathbf{B}^T\mathbf{C}\mathbf{B}Jw_p\right]\mathbf{U} = \mathbf{f}
$$
which can be written in a familiar form as
$$
\mathbf{F} = \mathbf{K}\mathbf{U}
$$